## First look at the data

In [ ]:
# this will make sure the root folder is the current working directory
from os import chdir, getcwd
from pyprojroot.here import here
os.chdir(here())
os.getcwd()

In [111]:
# libs
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# read data
df = pd.read_csv(
     "./data/01_input/Supermarket_customers.csv",
    delimiter="\t",              # note: \t because its a tab separated and not comma separated file
    parse_dates=["Dt_Customer"], # parse as dates
    dayfirst=True
) 

# transform columns for sane column names
df.columns = (df.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower()
             )

# remove absurd and yolo for now
df = df[~df["marital_status"].isin(["Absurd", "YOLO"])]

# replace alone -> single
df["marital_status"] = df["marital_status"].replace("Alone", "Single")

df.shape

In [166]:
def summarize_dataframe(df):
    # Container for all statistics
    all_stats = []

    for col in df.columns:
        col_data = df[col]
        num_missing = col_data.isna().sum()
        prop_missing = num_missing / len(col_data)

        if pd.api.types.is_datetime64_any_dtype(col_data):
            # Convert to numeric timestamps for calculations
            col_numeric = col_data.dropna().astype("int64")  # Dates as nanoseconds since epoch
            all_stats.append({
                "variable": col,
                "type": "datetime",
                "mean": pd.to_datetime(col_numeric.mean(), unit="ns") if not col_numeric.empty else None,
                "min": col_data.min(),
                "p05": col_data.quantile(0.05),
                "p50": col_data.median(),
                "p95": col_data.quantile(0.95),
                "max": col_data.max(),
                "num_missing": num_missing,
                "prop_missing": prop_missing
            })
        elif pd.api.types.is_numeric_dtype(col_data):
            all_stats.append({
                "variable": col,
                "type": "numeric",
                "mean": round(col_data.mean(), 4) if not col_data.empty else None,
                "min": col_data.min(),
                "p05": col_data.quantile(0.05),
                "p50": col_data.median(),
                "p95": col_data.quantile(0.95),
                "max": col_data.max(),
                "num_missing": num_missing,
                "prop_missing": prop_missing
            })
        elif isinstance(col_data.dtype, pd.CategoricalDtype) or col_data.dtype == "object":
            all_stats.append({
                "variable": col,
                "type": "categorical",
                "mean": None,
                "min": None,
                "p05": None,
                "p50": None,
                "p95": None,
                "max": None,
                "num_unique": col_data.nunique(),
                "num_missing": num_missing,
                "prop_missing": prop_missing
            })

    # Convert list of dictionaries to a DataFrame
    summary_df = pd.DataFrame(all_stats)
    return summary_df


In [ ]:
s1  = summarize_dataframe(df)
s1

In [168]:
## regress income on regressors to estimate missing income

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Define the list of predictor variables (with categorical columns included)
regressors = ["year_birth", "education", "marital_status", "kidhome", "teenhome",
              "recency", "mnt_wines", "mnt_fruits", "mnt_meat_products", "mnt_fish_products",
              "mnt_sweet_products", "mnt_gold_prods"]

# Separate rows with missing and non-missing income
df_missing = df[df["income"].isna()]
df_non_missing = df.dropna(subset=["income"])

# One-hot encode the categorical variables
df_non_missing_encoded = pd.get_dummies(df_non_missing[regressors], drop_first=True)
df_missing_encoded = pd.get_dummies(df_missing[regressors], drop_first=True)
df_all_obs_encoded = pd.get_dummies(df[regressors], drop_first=True)

# Ensure the training and missing data have the same columns (in case of mismatched categories)
df_non_missing_encoded, df_missing_encoded = df_non_missing_encoded.align(df_missing_encoded, join="left", axis=1, fill_value=0)

# Train a linear regression model on non-missing data
X_train = df_non_missing_encoded
y_train = df_non_missing["income"]
model = LinearRegression()
model.fit(X_train, y_train)

# Predict missing values
predicted_income = model.predict(df_missing_encoded)

# Impute the missing values in the original dataframe
# df.loc[df["income"].isna(), "income"] = predicted_income

# predict full (for comparison with available info)
predicted_income_full_df = model.predict(df_all_obs_encoded)
df["income_pred"] = predicted_income_full_df

df.to_csv("data/02_inter/tmp.csv")

In [ ]:
# get correlations 
df_full_encoded = pd.get_dummies(df, drop_first=True)
df_corr = df_full_encoded.corr()

# heatmap
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(
    round(df_corr,2), 
    cmap='RdBu',
    annot=True,
    annot_kws={"size": 5},
    vmin=-1, vmax=1,
    cbar=False
)

plt.show()

In [170]:
def summary_stats_by_group(df, group_var, target_var):
    
    # Calculate the summary statistics
    summary = df.groupby(group_var)[target_var].agg(
        count="count",  # Count the number of occurrences
        mean="mean",
        min="min",
        max="max",
        p01=lambda x: x.quantile(0.01),
        p05=lambda x: x.quantile(0.05),
        p10=lambda x: x.quantile(0.10),
        p25=lambda x: x.quantile(0.25),
        p50=lambda x: x.quantile(0.50),
        p75=lambda x: x.quantile(0.75),
        p90=lambda x: x.quantile(0.90),
        p95=lambda x: x.quantile(0.95),
        p99=lambda x: x.quantile(0.99)
    ).reset_index()

    # Sort by mean value (descending order)
    summary_sorted = summary.sort_values(by="mean", ascending=True)

    return summary_sorted

In [ ]:
if "mnt_total" not in df.columns: df["mnt_total"] = df.filter(like="mnt").sum(axis=1)
if "num_total" not in df.columns: df["num_total"] = df.filter(like="num").sum(axis=1)

# if cond so that does not run more than once
df.filter(regex="mnt|num").head()

In [ ]:
# summary by education, target variable: income
summary_stats_by_group(df, group_var="education", target_var="income")

In [ ]:
# summary by education, target variable: mnt_total
summary_stats_by_group(df, group_var="education", target_var="mnt_total")

In [ ]:
# summary by education, target variable: income
summary_stats_by_group(df, group_var="education", target_var="num_total")

In [ ]:
# summary by education, target variable: income
summary_stats_by_group(df, group_var="marital_status", target_var="income")

In [ ]:
# summary by education, target variable: mnt_total
summary_stats_by_group(df, group_var="marital_status", target_var="mnt_total")

In [ ]:
# summary by education, target variable: num_total
summary_stats_by_group(df, group_var="marital_status", target_var="num_total")

In [178]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


def plot_violin_strip(df, group_var, target_var, figsize=(6,6), jitter=True, cut=0):
    # Create a figure for the plot
    plt.figure(figsize=figsize)

    # Create the violin plot (horizontal)
    ax = sns.violinplot(
        x=target_var, y=group_var, data=df, color="lightblue", cut=cut, split=True
    )
    ax.set(xlabel=None, ylabel=None)

    # Title and labels
    plt.title(f"Distribution of {target_var} \nby {group_var}", loc="left")

    # Show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
plot_violin_strip(df, group_var="marital_status", target_var="mnt_total")
plot_violin_strip(df, group_var="education", target_var="mnt_total")

In [188]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_selected_features(df, features, target_var=None, nrows=5, ncols=5, figsize=(20, 20)):
    """
    Visualizes selected features of a DataFrame grouped by a target variable.
    
    Parameters:
    - df (pd.DataFrame): The input dataset.
    - features (list): A list of feature names to visualize.
    - target_var (str): The column name of the target variable
    - nrows (int): Number of rows in the subplot grid. Default is 5.
    - ncols (int): Number of columns in the subplot grid. Default is 5.
    - figsize (tuple): Size of the overall figure. Default is (20, 20).
    """
    # Define the figure and axes for the grid of subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    axes = axes.flatten()
    

    
    # Loop over each feature and plot based on its type
    for i, feature in enumerate(features):
        if i >= len(axes):  # Skip plotting if axes are exhausted
            break
        
        if df[feature].dtype in ["float64", "int64"] and df[feature].max() > 5:  # Numerical features
            sns.histplot(data=df, x=feature, hue=target_var, kde=True, ax=axes[i])

        elif df[feature].dtype in ["category", "object"]:  # Nominal features
            sns.countplot(data=df, x=feature, hue=target_var, ax=axes[i])

        else:  # Ordinal or other types
            sns.violinplot(data=df, x=target_var, y=feature, ax=axes[i])
        
        # Set the title for the plot
        axes[i].set_title(feature)
        # axes[i].set_yscale("log")  
        axes[i].set(xlabel=None, ylabel=None)
    
    # Hide any remaining unused axes in the grid
    for j in range(len(features), len(axes)):
        fig.delaxes(axes[j])
    

    plt.tight_layout()
    plt.show()

In [ ]:

num_features = [
    "mnt_wines",
    "mnt_fruits",
    "mnt_meat_products",
    "mnt_fish_products",
    "mnt_sweet_products",
    "mnt_gold_prods",
    "num_deals_purchases", 
    "num_web_purchases", 
    "num_catalog_purchases", 
    "num_store_purchases", 
    "num_web_visits_month"
]

# Call the function for specific features
visualize_selected_features(
    df=df,
    features=num_features,
    nrows=4, 
    ncols=3, 
    figsize=(8,8)
)